In [11]:
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，或在Jupyter notebook中!pip安装，参考上述环境配置
#!pip install pandas numpy cv2 torch torchvision codecs PIL glob
#---------------------------------------------------
#导入库
import os
import glob
from PIL import Image
import csv, time
import numpy as np

# pytorch相关
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.utils.data as data
import albumentations as A

In [12]:
# 自定义读取数据集
class ImageSet(data.Dataset):
    def __init__(
            self,
            images,
            labels,
            transform):
        self.transform = transform
        self.images = images
        self.labels = labels

    def __getitem__(self, item):
        imagename = self.images[item]
        
        # 防止文件出错，这里生成一个随机的照片
        try:
            image = Image.open(imagename)
            image = image.convert('RGB')
        except:
            image = Image.fromarray(np.zeros((256, 256), dtype=np.int8))
            image = image.convert('RGB')

        image = self.transform(image)
        return image, torch.tensor(self.labels[item])

    def __len__(self):
        return len(self.images)

In [13]:
import pandas as pd
import codecs

# 训练集标注数据
lines = codecs.open('train_label.csv').readlines()
train_label = pd.DataFrame({
    'image': ['train_image/' + x.strip().split(',')[0] for x in lines],
    'label': [x.strip().split(',')[1:] for x in lines],
})

# 将标签进行二值化处理
train_label['new_label'] = train_label['label'].apply(lambda x: int('0' in x))

# 数据扩增方法
trfs = transforms.Compose([
    A.RandomCrop(450, 750),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    A.CoarseDropout(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 训练集dataset和dataloder
# 这里我们使用前1000张图片进行训练，后续可以自行修改
train_dataset = ImageSet(train_label['image'].values,
                         train_label['new_label'].values,
                         trfs)
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

# 测试集dataset和dataloder
test_images = glob.glob('./test_images/*')
test_dataset = ImageSet(test_images, [0] * len(test_images), trfs)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

In [14]:
from PIL import Image
from torchvision.datasets import DatasetFolder
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from tqdm.auto import tqdm

batch_size = 128

unlabeled_set = DatasetFolder(r"C:\Users\BiXY\OneDrive - 厦门大学(马来西亚分校)\竞赛\华为AI算法赛\digix-2022-cv-sample\digix-2022-cv-sample\train_image\unlabeled_data", loader=lambda x: Image.open(x).convert('RGB'), extensions="png", transform=trfs)

class PseudoDataset(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.label = y

    def __getitem__(self, idx): 
        return self.data[idx][0], torch.tensor(self.label[idx])
        

    def __len__(self):
        return len(self.label)
    
def get_pseudo_labels(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)
    # Iterate over the dataset by batches.
    new_sample = []
    new_label = []
    #i = 0
    for batch in tqdm(data_loader):
        img, _ = batch
        #print("****img:",img.shape)
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)

        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        probs_argmax = probs.argmax(dim=-1).cpu().numpy().tolist()
        probs_max = probs.max(dim=-1).values
        probs_max = probs_max.cpu().numpy().tolist()
        #feat = []
        i = 0
        for idx,p in enumerate(probs_max):
            if p > threshold:
                new_sample.append(idx + i * batch_size)
                new_label.append(probs_argmax[idx])
        i = i + 1
    new_data = Subset(dataset,new_sample)          
    new_label_set = PseudoDataset(new_data,new_label)
    # # Turn off the eval mode.
    model.train()
    return new_label_set

In [15]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
# Initialize a model, and put it on the device specified.
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
train_dataset = ImageSet(train_label['image'].values,
                         train_label['new_label'].values,
                         trfs)
print(train_dataset.__getitem__(0))

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [20]:
class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        # 此处的self.smoothing即我们的epsilon平滑参数。

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [ ]:
# 加载resnet18预训练模型
model = torchvision.models.resnet34(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to('cuda') #使用GPU

# 模型优化器
optimizer = optim.SGD(model.parameters(), lr=0.001)

# 模型损失函数
loss = LabelSmoothing(smoothing=0.1)

# 接下来我们使用pytorch完成模型的训练：
# Python
# 设置迭代轮数epochs，可调整，轮数越多，所花时间越久
epochs = 2
for epoch in range(epochs):
    start_t = time.time()
    epoch_l = 0
    epoch_t = 0
    pseudo_set = get_pseudo_labels(unlabeled_set, model)

    # Construct a new dataset and a data loader for training.
    # This is used in semi-supervised learning only.
    concat_dataset = ConcatDataset([train_dataset, pseudo_set])
    train_loader = DataLoader(concat_dataset, batch_size=32, shuffle=True, num_workers=0)
    
    # 批量训练
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        image, label = batch
        image, label = image.to('cuda'), label.to('cuda')
        output = model(image) # 正向传播

        l = loss(output, label) # 计算损失
        l.backward()
        optimizer.step()

        batch_l = l.item()
        epoch_l += batch_l
        batch_t = time.time() - start_t
        epoch_t += batch_t
        start_t = time.time()
        
        # 打印loss
        if batch_idx % 10 == 0:
            print(l.item(), batch_idx, len(train_loader))

    epoch_t = epoch_t / len(train_loader)
    epoch_l = epoch_l / len(train_loader)
    print('...epoch: {:3d}/{:3d}, loss: {:.4f}, average time: {:.2f}.'.format(
        epoch + 1, epochs, epoch_l, epoch_t))

100%|████████████████████████████████████████████████████████████████████████████| 1036/1036 [1:43:37<00:00,  6.00s/it]


0.35695362091064453 0 4013
0.3913387060165405 10 4013
0.22286942601203918 20 4013
0.21221011877059937 30 4013
0.21745653450489044 40 4013
0.21446934342384338 50 4013
0.26699262857437134 60 4013
0.2683424949645996 70 4013
0.2088620662689209 80 4013
0.3177863359451294 90 4013
0.2615867853164673 100 4013
0.21865057945251465 110 4013
0.21652933955192566 120 4013
0.26618847250938416 130 4013
0.30943745374679565 140 4013
0.2704204022884369 150 4013
0.26976990699768066 160 4013
0.21105660498142242 170 4013
0.25849518179893494 180 4013
0.2205537110567093 190 4013
0.2593487501144409 200 4013
0.3708856701850891 210 4013
0.31158629059791565 220 4013
0.26527106761932373 230 4013
0.2506088614463806 240 4013
0.2627578377723694 250 4013
0.21094045042991638 260 4013
0.2273663878440857 270 4013
0.2091745138168335 280 4013
0.20608395338058472 290 4013
0.2605726718902588 300 4013
0.20600956678390503 310 4013
0.21832594275474548 320 4013
0.20664499700069427 330 4013
0.35769402980804443 340 4013
0.21110926

0.22025105357170105 2820 4013
0.2557830810546875 2830 4013
0.2564833164215088 2840 4013
0.20998087525367737 2850 4013
0.2604927718639374 2860 4013
0.2596915364265442 2870 4013
0.21062573790550232 2880 4013
0.2114545702934265 2890 4013
0.3687609136104584 2900 4013
0.26283690333366394 2910 4013
0.25771141052246094 2920 4013
0.3140549659729004 2930 4013
0.26895833015441895 2940 4013
0.20522922277450562 2950 4013
0.3762352764606476 2960 4013
0.30571043491363525 2970 4013
0.26241540908813477 2980 4013
0.20565734803676605 2990 4013
0.20458991825580597 3000 4013
0.20450067520141602 3010 4013
0.260669469833374 3020 4013
0.2075699269771576 3030 4013
0.3108227550983429 3040 4013
0.2082522064447403 3050 4013
0.20517602562904358 3060 4013
0.31860530376434326 3070 4013
0.25850751996040344 3080 4013
0.3132564127445221 3090 4013
0.2583811581134796 3100 4013
0.2620445787906647 3110 4013
0.3906550407409668 3120 4013
0.25535768270492554 3130 4013
0.20776313543319702 3140 4013
0.21002864837646484 3150 40

 49%|██████████████████████████████████████▋                                        | 508/1036 [55:07<57:36,  6.55s/it]

In [ ]:
model.eval()
to_prob = nn.Softmax(dim=1)
with torch.no_grad():
    imagenames, probs = list(), list()
    for batch_idx, batch in enumerate(test_loader):
        image, _ = batch
        image = image.to('cuda')
        pred = model(image)
        prob = to_prob(pred)
        prob = list(prob.data.cpu().numpy())
        probs += prob

In [ ]:
import csv
with open('submission.csv', 'w',newline = '', encoding='utf8') as fp:
    writer = csv.writer(fp)
    writer.writerow(['imagename', 'defect_prob'])
    for imagename, prob in zip(test_images, probs):
        imagename = os.path.basename(imagename)
        writer.writerow([imagename, str(prob[1])])